In [17]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')
car=pd.read_csv('car_price.csv')
car.head()

,Unnamed: 0,car_name,car_prices_in_rupee,kms_driven,fuel_type,transmission,ownership,manufacture,engine,Seats
0,0,Jeep Compass 2.0 Longitude Option BSIV,10.03 Lakh,"86,226 kms",Diesel,Manual,1st Owner,2017,1956 cc,5 Seats
1,1,Renault Duster RXZ Turbo CVT,12.83 Lakh,"13,248 kms",Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats
2,2,Toyota Camry 2.5 G,16.40 Lakh,"60,343 kms",Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats
3,3,Honda Jazz VX CVT,7.77 Lakh,"26,696 kms",Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats
4,4,Volkswagen Polo 1.2 MPI Highline,5.15 Lakh,"69,414 kms",Petrol,Manual,1st Owner,2016,1199 cc,5 Seats


1.Mô tả chi tiết từng thuộc tính

car_name: Tên xe, kiểu string
car_prices_in_rupee: Giá xe, kiểu string
kms_driven: Số km đã đi, kiểu string
fuel_type: Kiểu nhiên liệu, kiểu string
transmission: Hộp số, kiểu string
ownership: Số người sở hữu, kiểu string
manufacture: Năm sản xuất, kiểu int
engine: Động cơ, kiểu string
Seats: Số ghế, kiểu string


In [18]:

car.shape


(5512, 10)

In [19]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5512 entries, 0 to 5511
Data columns (total 10 columns):
Unnamed: 0             5512 non-null int64
car_name               5512 non-null object
car_prices_in_rupee    5512 non-null object
kms_driven             5512 non-null object
fuel_type              5512 non-null object
transmission           5512 non-null object
ownership              5512 non-null object
manufacture            5512 non-null int64
engine                 5512 non-null object
Seats                  5512 non-null object
dtypes: int64(2), object(8)
memory usage: 430.7+ KB


2.MÔ TẢ CHI TIẾT KIỂU DỮ LIỆU CỦA THUỘC TÍNH CẦN DỰ ĐOÁN

- car_name: Tên xe, kiểu string 
=> manufacturer: Tên nhà sx; model: Mô hình xe, kiểu string
- car_prices_in_rupee: Giá xe, kiểu string
- kms_driven: Số km đã đi, kiểu string
- transmission: Hộp số, kiểu string
- manufacture: Năm sản xuất, kiểu int
- engine: Động cơ, kiểu string
- Seats: Số ghế, kiểu string


In [20]:
#Tạo bản sao lưu
backup=car.copy()
backup.head()

,Unnamed: 0,car_name,car_prices_in_rupee,kms_driven,fuel_type,transmission,ownership,manufacture,engine,Seats
0,0,Jeep Compass 2.0 Longitude Option BSIV,10.03 Lakh,"86,226 kms",Diesel,Manual,1st Owner,2017,1956 cc,5 Seats
1,1,Renault Duster RXZ Turbo CVT,12.83 Lakh,"13,248 kms",Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats
2,2,Toyota Camry 2.5 G,16.40 Lakh,"60,343 kms",Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats
3,3,Honda Jazz VX CVT,7.77 Lakh,"26,696 kms",Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats
4,4,Volkswagen Polo 1.2 MPI Highline,5.15 Lakh,"69,414 kms",Petrol,Manual,1st Owner,2016,1199 cc,5 Seats


In [21]:
#Loại bỏ cột Unname
car = car.drop(columns=["Unnamed: 0"], axis=1)

In [22]:
#Đổi tên cột trong data farme
car.rename(columns={'car_name': 'model'}, inplace=True)
#Tạo ra cột mới, giá trị chưa tên nhà sx được trích xuất từ cột model
car['manufacturer'] = car['model'].map(lambda x: str(x).strip().split(" ")[0])
#Tạo ra cột model mới mỗi giá trị sẽ chưa tên mô hình xe
car['model'] = car['model'].map(lambda x: str(x).strip().split(" ")[1])


3.VIẾT HÀM CHUYỂN ĐỔI SANG SỐ

In [23]:
#Loại bỏ kí tự km và khoảng trắng
car['kms_driven'] = car['kms_driven'].map(lambda x: x.rstrip(' kms'))
# thay thế dấu phẩy bằng một chuỗi rỗng
car['kms_driven'] = car['kms_driven'].map(lambda x: x.replace(',', ''))
#chuyển đổi giá trị trong cột 'kms_driven' thành kiểu dữ liệu số
car['kms_driven'] = pd.to_numeric(car['kms_driven'])

In [24]:
#loại bỏ các khoảng trắng và ký tự 'cc'
car['engine'] = car['engine'].map(lambda x: x.rstrip(' cc'))
#chuyển đổi giá trị trong cột 'engine' thành kiểu dữ liệu số
car['engine'] = pd.to_numeric(car['engine'])

In [25]:
# Loại bỏ từ Seat
car['Seats'] = car['Seats'].map(lambda x: x.rstrip(' Seats'))

In [26]:
#đếm số lượng xuất hiện của mỗi nhà sản xuất
car['manufacturer'].value_counts()

Maruti           1389
Hyundai          1023
Honda             498
Mahindra          315
Mercedes-Benz     283
Tata              269
Toyota            263
BMW               172
Renault           170
Ford              168
Audi              166
Volkswagen        159
Skoda             113
Chevrolet          82
Nissan             63
MG                 62
Kia                61
Land               50
Jeep               49
Volvo              31
Jaguar             30
Datsun             25
Fiat               17
Mini               14
Porsche            10
Lexus              10
Mitsubishi          8
Maserati            5
Isuzu               4
Bentley             1
Premier             1
Force               1
Name: manufacturer, dtype: int64

In [27]:
#tách cột 'car_prices_in_rupee' thành hai cột mới là 'price' và 'multiply' trong DataFrame 
car[['price', 'multiply']] = car['car_prices_in_rupee'].str.split(' ', 1, expand=True)
# loại bỏ cột 'car_prices_in_rupee' khỏi DataFrame
car.drop('car_prices_in_rupee', axis=1, inplace=True)

In [28]:
#được sử dụng để thay thế chuỗi 'Lakh' bằng giá trị số 100000 trong cột 'multiply' của DataFrame 
car['multiply']=car['multiply'].str.replace('Lakh','100000')
# thay thế chuỗi 'Crore' bằng giá trị số 10000000 trong cột 'multiply' của DataFrame
car['multiply']=car['multiply'].str.replace('Crore','10000000')
#chuyển đổi giá trị trong cột 'multiply' thành kiểu dữ liệu số.
car['multiply'] = pd.to_numeric(car['multiply'])


0    100000.0
1    100000.0
2    100000.0
3    100000.0
4    100000.0
Name: multiply, dtype: float64

In [29]:
car['price']=car['price'].str.replace(',','')
car['price'] = pd.to_numeric(car['price'])
car['price'] = car['price'] * car['multiply']

In [30]:
#Loại bỏ cột multiply
car = car.drop(columns=["multiply"], axis=1)